In [1]:
import os, sys
sys.path.append('/home/oleg/Prog/Science/ship_tracks_recognition/functions')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from math import *
from Y_views import *
from UV_views import *
%matplotlib inline

In [3]:
all_mctracks = pd.read_csv('Data/MCTrack.csv', delimiter=',')
all_velo_points = pd.read_csv('Data/vetoPoint.csv', delimiter=',')
all_hits = pd.read_csv('Data/strawtubesPoint_decoded_hits.csv', delimiter=',')
all_hits['Index'] = all_hits.index

In [ ]:
from ReconstructibleTracks import get_reconstractible_tracks

reconstructible_events = {}

for event_id in range(0, 2500):
    
    tracks = get_reconstractible_tracks(event_id, all_hits, all_mctracks, all_velo_points)
    
    if tracks != []:
        reconstructible_events[event_id] = tracks

In [ ]:
reco = np.array(reconstructible_events.keys())
reco

In [ ]:
pd.DataFrame(reco).to_csv('./Data/RecognizedTracks.csv')

In [4]:
reco = np.array(pd.read_csv('./Data/RecognizedTracks.csv'))[:, 1]
reco

array([ 513,    2,  515,  516, 1541,  518,    7,   10, 2059, 1036, 1550,
         15, 2392, 1042,  686,   23, 2360,   27, 2077, 1057,  549,   39,
       2271, 1066, 1655, 2482,  558, 2095,   48,   50, 2099,   53, 1204,
       1546,   62, 1775, 1600, 2113, 1090, 2423, 1206, 1606, 1607, 1548,
         74, 1293, 1213,   84,   85,   86, 1111, 2138, 1116, 1629,  606,
       1637,  608,   97, 1979,  612,  102,  614, 1129,  106, 1895, 2305,
       2157,  622,  957, 1136,  625, 1138, 2232, 2164, 1893, 1142, 1385,
       2493,  122, 1668,  126, 1557,  129,  642,  131,  644, 1157,  134,
        135,  648, 1280,  651, 1474,  654,  143, 1683, 2224, 1691, 1180,
        157,  670,  159, 2208,  161, 2246, 1190, 1704,  174, 1200, 1224,
        179,  692, 1717,  694,  184, 1722, 1724,  189, 2239, 1677, 1440,
        707, 2244, 1967, 1222,  712,  715, 1900,  719, 1232, 2446,  211,
       2260, 1238,  215, 2264, 1242, 1243, 1756,  221, 1758,  735,  736,
       1761, 1762, 1251,  806, 1256, 1769, 2282,  7

In [5]:
from sklearn import cross_validation
train_events, test_events = cross_validation.train_test_split(reco, test_size=0.5)

# Without classifier

In [6]:
all_events = len(test_events)
recognized_events = 0
ind = 0
n_min = 7
plane_width = 0.85
for i in test_events:
    event = all_hits[all_hits.event==i]
    tracks, linking_table = loop_yz(event, n_min, plane_width, ind)
    if len(tracks)>1:
        recognized_events += 1
print 1.0 * recognized_events / all_events

0.981132075472


/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Train classifier

In [18]:
table = pd.read_csv('./Data/TracksTable.csv')

In [19]:
table = table[table['event'].isin(test_events)]
table = table[['N', 'Max', 'Min', 'deviation', 'Class']]

In [20]:
train_table, test_table = cross_validation.train_test_split(table, train_size=0.8)

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, class_weight='auto')

In [22]:
clf.fit(train_table[['N', 'Max', 'Min', 'deviation']], train_table['Class'])

RandomForestClassifier(bootstrap=True, class_weight='auto', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
clf.score(test_table[['N', 'Max', 'Min', 'deviation']], test_table['Class'])

0.97619047619047616

# With classifier

In [24]:
from Y_views_new import *

In [25]:
all_events = len(test_events)
recognized_events = 0
ind = 0
n_min = 7
plane_width = 0.85
for i in test_events:
    event = all_hits[all_hits.event==i]
    tracks, linking_table = loop_yz_new(event, n_min, plane_width, ind, clf)
    if len(tracks)>1:
        recognized_events += 1
print 1.0 * recognized_events / all_events

0.962264150943
